In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
import copy
# import data_loader
import pdb
import torch.nn.functional as F
import re, random, collections
import pickle


import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='3'

In [2]:
class args():

    data_path = "./Datasets/CIFAR100/"
    num_class = 100
    class_per_task = 20
    num_task = 5
    dataset = "cifar100"
    train_batch = 128
    test_batch = 128
    workers = 16
    
    random_classes = False
    validation = 0
    overflow=False
    
    batch_size=128
    lr=0.01
    resume=False
    total_epoch=250
    model_path='ckpt816/cifar100-5'
    
args=args()

In [3]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    
# seed=torch.randint(1,10000,[1])   
set_seed(3473)

In [4]:
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import argparse
# from utils import progress_bar
from torch.optim.lr_scheduler import MultiStepLR
torch.set_printoptions(precision=5,sci_mode=False)

In [5]:
class ConvAdapt(nn.Module):
    def __init__(self, in_channels, out_channels, p):
        super(ConvAdapt, self).__init__()
        # Groupwise Convolution
        self.gwc = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, groups=int(p/gp), bias=True)
        # Pointwise Convolution
        self.pwc = nn.Conv2d(in_channels, out_channels, kernel_size=1, groups=int(p/pt),bias=True)

    def forward(self, x):
        return self.gwc(x) + self.pwc(x)

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.lhc1=ConvAdapt(planes,planes,planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.lhc2=ConvAdapt(planes,planes,planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                ConvAdapt(self.expansion*planes,self.expansion*planes,int(self.expansion*planes)),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.lhc1(self.conv1(x))))
        out = self.bn2(self.lhc2(self.conv2(out)))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=args.class_per_task):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.hc1=ConvAdapt(64,64,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out=self.conv1(x)
        out = F.relu(self.bn1(out))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        feat = out.view(out.size(0), -1)
        out = self.linear(feat)
        return out,feat


def Net():
    return ResNet(BasicBlock, [2,2,2,2])


In [6]:
def save_model(task,acc,model):
    print('Saving..')
    statem = {
        'net': model.state_dict(),
        'acc': acc,
    }
    fname=args.model_path
    if not os.path.isdir(fname):
        os.makedirs(fname)
    torch.save(statem, fname+'/ckpt_task'+str(task)+'.pth')

        
def load_model(task,model):
    fname=args.model_path
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    print(fname+'/ckpt_task'+str(task)+'.pth')
    checkpoint = torch.load(fname+'/ckpt_task'+str(task)+'.pth')
    model.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    return best_acc

In [7]:
def train(train_loader,epoch,task,model,task_model):
    print('\nEpoch: %d' % epoch)
    model.train()
    global best_acc
    train_loss = 0
    correct = 0
    total = 0

    Sloss=0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        targets=targets-task*args.class_per_task # assume sequential split for random  split mapping should me changed
        optimizer.zero_grad()
        outputs,feat_current = model(inputs)
 
        loss = criterion(outputs, targets)
    
        ################################################################################
        ############# this is slow version here we dont save \mu and \sigma of KLD######
        # saving mu and sigma and maximize the distance from the current batch significantly 
        # speedup the model####
        if task>=1:
            for m in range(task):
                with torch.no_grad():
                    task_model[m].eval()
                    out,feat_prev=task_model[m](inputs)
                mean_loss=torch.max(1-torch.dist(feat_current,feat_prev,2)/args.train_batch,
                                    torch.zeros(1).cuda())/task
                loss+=0.05*mean_loss[0]
        ###################################################################################

        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100.*correct/total
    
    print("[Train: ], [%d/%d: ], [Accuracy: %f], [Loss: %f] [LossS: %f] [Lr: %f]" 
          %(epoch,args.total_epoch,acc, train_loss/batch_idx,Sloss/batch_idx,
            optimizer.param_groups[0]['lr']))

In [8]:
def check_task(task,inputs,model,task_model):
    joint_entropy=[]
    with torch.no_grad():
        for m in range(task):
            task_model[m].eval()
            outputs,_=task_model[m](inputs)

            sout=F.softmax(outputs,1)
            entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
            joint_entropy.append(entropy)

        model.eval()
        outputs,_=model(inputs)

        sout=F.softmax(outputs,1)
        entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
        joint_entropy.append(entropy)


    all_entropy=torch.zeros([joint_entropy[0].shape[0],task+1]).cuda()
    for i in range(task+1):
        all_entropy[:,i]=joint_entropy[i]
    ctask=torch.argmin(all_entropy, axis=1)==task
    correct=sum(ctask)
    
    return ctask, correct,all_entropy

    

In [9]:
def test(test_loader,task,model,task_model):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    cl_loss=0
    tcorrect=0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets1 = inputs.cuda(), targets.cuda()
            targets=targets1-task*args.class_per_task
            if task>0:
                correct_sample,Ncorrect,_=check_task(task,inputs,model,task_model)
                tcorrect+=Ncorrect
                inputs=inputs[correct_sample]
                targets=targets[correct_sample]
            if inputs.shape[0]!=0:
                outputs,_ = model(inputs)
                loss = criterion(outputs, targets)


                test_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()
            total += targets1.size(0)
    if task>0:
        taskC= tcorrect.item()/total
    else:
        taskC=1.0 
    acc = 100.*correct/total
    print("[Test Accuracy: %f], [Loss: %f] [Correct: %f]" %(acc,test_loss/batch_idx,taskC))
    
    
    if acc>=best_acc:
        save_model(task,acc,model)
        best_acc=acc
    return acc

In [10]:
# this false the gradient of the global parameter after the 1st task

def grad_false(model):
    gradf=[0,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133]
    i=0
    for p in model.parameters():
        if i in gradf:
            p.requires_grad=False
        i=i+1

In [11]:
import incremental_dataloader as data
inc_dataset = data.IncrementalDataset(
                                dataset_name=args.dataset,
                                args = args,
                                random_order=args.random_classes,
                                shuffle=True,
                                seed=1,
                                batch_size=args.train_batch,
                                workers=args.workers,
                                validation_split=args.validation,
                                increment=args.class_per_task,
                            )
task_data=[]
for i in range(args.num_task):
    task_info, train_loader, val_loader, test_loader = inc_dataset.new_task()

    task_data.append([train_loader,test_loader])

Files already downloaded and verified
Files already downloaded and verified
0
[20, 20, 20, 20, 20]
1
[20, 20, 20, 20, 20]
2
[20, 20, 20, 20, 20]
3
[20, 20, 20, 20, 20]
4
[20, 20, 20, 20, 20]


In [ ]:
# groupwise and pointwise group convolutional size
gp=8
pt=16
###############################################

task_model=[]
task_acc=[]
criterion = nn.CrossEntropyLoss()

for task in range(args.num_task):
    if task==0:
        best_acc=0
        print('Training Task :---'+str(task))

        train_loader, test_loader = task_data[task][0],task_data[task][1]
        modelm = Net().cuda()

#         criterion = nn.CrossEntropyLoss()
        mse_loss=nn.MSELoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-3)
        schedulerG = MultiStepLR(optimizer, milestones=[100, 150, 200], gamma=0.1)

        for epoch in range(args.total_epoch):
            train(train_loader,epoch,task,modelm,task_model)
            test(test_loader,task,modelm,task_model)
            schedulerG.step()
        acc1=load_model(task,modelm)
        task_model.append(copy.deepcopy(modelm))


    if task!=0:
        print('Training Task :---'+str(task))
        best_acc=0
        train_loader, test_loader = task_data[task][0],task_data[task][1]
        modelm = Net().cuda()

        acc1=load_model(task-1,modelm)  
        grad_false(modelm)

#         criterion = nn.CrossEntropyLoss()
        mse_loss=nn.MSELoss()
        optimizer = optim.SGD(modelm.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-3)
        schedulerG = MultiStepLR(optimizer, milestones=[100,150,200],gamma=0.1)
        if task>=1:
            for epoch in range(args.total_epoch):
                train(train_loader,epoch,task,modelm,task_model)
                test(test_loader,task,modelm,task_model)
                schedulerG.step()

        acc1=load_model(task,modelm)
        task_model.append(copy.deepcopy(modelm))

    task_acc.append(acc1)
    print('Task: '+str(task)+'  Test_accuracy: '+ str(acc1))

# This evaluation only check the accuracy of the current task with all the previous task.
# therefore not correct since if trained the model for the 10 task the 1st task will get 
# evaluated with 10 task. Next cell provides the model evaluation

print(task_acc)


Training Task :---0

Epoch: 0
[Train: ], [0/250: ], [Accuracy: 21.580000], [Loss: 2.591609] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 26.400000], [Loss: 2.872587] [Correct: 1.000000]
Saving..

Epoch: 1
[Train: ], [1/250: ], [Accuracy: 33.370000], [Loss: 2.215690] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 36.050000], [Loss: 2.178962] [Correct: 1.000000]
Saving..

Epoch: 2
[Train: ], [2/250: ], [Accuracy: 40.140000], [Loss: 1.979313] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 39.050000], [Loss: 2.155543] [Correct: 1.000000]
Saving..

Epoch: 3
[Train: ], [3/250: ], [Accuracy: 44.980000], [Loss: 1.791175] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 39.650000], [Loss: 2.240297] [Correct: 1.000000]
Saving..

Epoch: 4
[Train: ], [4/250: ], [Accuracy: 49.060000], [Loss: 1.652335] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 49.850000], [Loss: 1.784058] [Correct: 1.000000]
Saving..

Epoch: 5
[Train: ], [5/250: ], [Accuracy: 52.370000], [Loss: 1.531913] [LossS: 0.000

[Train: ], [47/250: ], [Accuracy: 85.470000], [Loss: 0.472168] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 73.050000], [Loss: 0.977092] [Correct: 1.000000]

Epoch: 48
[Train: ], [48/250: ], [Accuracy: 85.550000], [Loss: 0.468029] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 71.950000], [Loss: 0.990242] [Correct: 1.000000]

Epoch: 49
[Train: ], [49/250: ], [Accuracy: 85.920000], [Loss: 0.458069] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 75.450000], [Loss: 0.831815] [Correct: 1.000000]

Epoch: 50
[Train: ], [50/250: ], [Accuracy: 86.330000], [Loss: 0.437952] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 73.800000], [Loss: 0.927832] [Correct: 1.000000]

Epoch: 51
[Train: ], [51/250: ], [Accuracy: 85.920000], [Loss: 0.460726] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 73.400000], [Loss: 0.962090] [Correct: 1.000000]

Epoch: 52
[Train: ], [52/250: ], [Accuracy: 85.310000], [Loss: 0.466486] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 69.900000], [Loss: 1.051504]

[Test Accuracy: 77.600000], [Loss: 0.762001] [Correct: 1.000000]

Epoch: 95
[Train: ], [95/250: ], [Accuracy: 87.230000], [Loss: 0.417485] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 73.050000], [Loss: 0.967008] [Correct: 1.000000]

Epoch: 96
[Train: ], [96/250: ], [Accuracy: 89.970000], [Loss: 0.341269] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 76.650000], [Loss: 0.825232] [Correct: 1.000000]

Epoch: 97
[Train: ], [97/250: ], [Accuracy: 87.460000], [Loss: 0.412031] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 76.550000], [Loss: 0.857830] [Correct: 1.000000]

Epoch: 98
[Train: ], [98/250: ], [Accuracy: 87.890000], [Loss: 0.395583] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 77.500000], [Loss: 0.783983] [Correct: 1.000000]

Epoch: 99
[Train: ], [99/250: ], [Accuracy: 90.800000], [Loss: 0.309942] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 76.500000], [Loss: 0.822841] [Correct: 1.000000]

Epoch: 100
[Train: ], [100/250: ], [Accuracy: 95.540000], [Loss: 0.170979]

[Test Accuracy: 86.300000], [Loss: 0.479514] [Correct: 1.000000]

Epoch: 142
[Train: ], [142/250: ], [Accuracy: 99.780000], [Loss: 0.036848] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 86.050000], [Loss: 0.486181] [Correct: 1.000000]

Epoch: 143
[Train: ], [143/250: ], [Accuracy: 99.870000], [Loss: 0.037590] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 86.850000], [Loss: 0.476359] [Correct: 1.000000]

Epoch: 144
[Train: ], [144/250: ], [Accuracy: 99.800000], [Loss: 0.035442] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 86.750000], [Loss: 0.486728] [Correct: 1.000000]

Epoch: 145
[Train: ], [145/250: ], [Accuracy: 99.820000], [Loss: 0.038640] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 86.450000], [Loss: 0.480324] [Correct: 1.000000]

Epoch: 146
[Train: ], [146/250: ], [Accuracy: 99.830000], [Loss: 0.037558] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 86.500000], [Loss: 0.492844] [Correct: 1.000000]

Epoch: 147
[Train: ], [147/250: ], [Accuracy: 99.790000], [Loss:

[Test Accuracy: 86.800000], [Loss: 0.485942] [Correct: 1.000000]

Epoch: 189
[Train: ], [189/250: ], [Accuracy: 99.840000], [Loss: 0.033009] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 86.750000], [Loss: 0.479574] [Correct: 1.000000]

Epoch: 190
[Train: ], [190/250: ], [Accuracy: 99.890000], [Loss: 0.032014] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 86.500000], [Loss: 0.489554] [Correct: 1.000000]

Epoch: 191
[Train: ], [191/250: ], [Accuracy: 99.950000], [Loss: 0.030918] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 86.350000], [Loss: 0.483763] [Correct: 1.000000]

Epoch: 192
[Train: ], [192/250: ], [Accuracy: 99.950000], [Loss: 0.029795] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 87.050000], [Loss: 0.482213] [Correct: 1.000000]

Epoch: 193
[Train: ], [193/250: ], [Accuracy: 99.890000], [Loss: 0.032407] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 86.450000], [Loss: 0.485712] [Correct: 1.000000]

Epoch: 194
[Train: ], [194/250: ], [Accuracy: 99.940000], [Loss:

...

[Test Accuracy: 58.250000], [Loss: 0.859294] [Correct: 0.735500]

Epoch: 80
[Train: ], [80/250: ], [Accuracy: 87.210000], [Loss: 0.440422] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 58.000000], [Loss: 0.698355] [Correct: 0.701000]

Epoch: 81
[Train: ], [81/250: ], [Accuracy: 87.570000], [Loss: 0.429038] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 56.600000], [Loss: 0.761412] [Correct: 0.692000]

Epoch: 82
[Train: ], [82/250: ], [Accuracy: 88.240000], [Loss: 0.412825] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 61.650000], [Loss: 0.629983] [Correct: 0.735000]

Epoch: 83
[Train: ], [83/250: ], [Accuracy: 88.200000], [Loss: 0.409816] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 58.550000], [Loss: 0.682064] [Correct: 0.716500]

Epoch: 84
[Train: ], [84/250: ], [Accuracy: 88.040000], [Loss: 0.421697] [LossS: 0.000000] [Lr: 0.010000]
[Test Accuracy: 59.950000], [Loss: 0.628322] [Correct: 0.718000]

Epoch: 85
[Train: ], [85/250: ], [Accuracy: 88.260000], [Loss: 0.411513] [

[Test Accuracy: 74.650000], [Loss: 0.384744] [Correct: 0.826500]

Epoch: 127
[Train: ], [127/250: ], [Accuracy: 97.150000], [Loss: 0.148130] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 74.550000], [Loss: 0.400796] [Correct: 0.826500]

Epoch: 128
[Train: ], [128/250: ], [Accuracy: 97.430000], [Loss: 0.140342] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 74.150000], [Loss: 0.390650] [Correct: 0.826000]

Epoch: 129
[Train: ], [129/250: ], [Accuracy: 97.590000], [Loss: 0.137180] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 74.000000], [Loss: 0.396295] [Correct: 0.822500]

Epoch: 130
[Train: ], [130/250: ], [Accuracy: 97.560000], [Loss: 0.137422] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 74.700000], [Loss: 0.385874] [Correct: 0.823000]

Epoch: 131
[Train: ], [131/250: ], [Accuracy: 97.380000], [Loss: 0.133626] [LossS: 0.000000] [Lr: 0.001000]
[Test Accuracy: 74.200000], [Loss: 0.403552] [Correct: 0.823500]

Epoch: 132
[Train: ], [132/250: ], [Accuracy: 97.710000], [Loss:

[Test Accuracy: 75.000000], [Loss: 0.420947] [Correct: 0.829500]

Epoch: 174
[Train: ], [174/250: ], [Accuracy: 98.760000], [Loss: 0.099220] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 74.850000], [Loss: 0.400419] [Correct: 0.828500]

Epoch: 175
[Train: ], [175/250: ], [Accuracy: 98.660000], [Loss: 0.100367] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 74.100000], [Loss: 0.396713] [Correct: 0.816500]

Epoch: 176
[Train: ], [176/250: ], [Accuracy: 98.600000], [Loss: 0.099958] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 74.800000], [Loss: 0.420110] [Correct: 0.827000]

Epoch: 177
[Train: ], [177/250: ], [Accuracy: 98.820000], [Loss: 0.094633] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 74.900000], [Loss: 0.406242] [Correct: 0.827500]

Epoch: 178
[Train: ], [178/250: ], [Accuracy: 98.860000], [Loss: 0.098910] [LossS: 0.000000] [Lr: 0.000100]
[Test Accuracy: 74.850000], [Loss: 0.414025] [Correct: 0.829500]

Epoch: 179
[Train: ], [179/250: ], [Accuracy: 98.700000], [Loss:

[Test Accuracy: 75.550000], [Loss: 0.397134] [Correct: 0.833500]

Epoch: 221
[Train: ], [221/250: ], [Accuracy: 98.770000], [Loss: 0.093919] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 74.650000], [Loss: 0.399862] [Correct: 0.824500]

Epoch: 222
[Train: ], [222/250: ], [Accuracy: 98.650000], [Loss: 0.095894] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 75.300000], [Loss: 0.387281] [Correct: 0.828000]

Epoch: 223
[Train: ], [223/250: ], [Accuracy: 98.540000], [Loss: 0.103095] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 75.200000], [Loss: 0.391793] [Correct: 0.832000]

Epoch: 224
[Train: ], [224/250: ], [Accuracy: 98.780000], [Loss: 0.102669] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 75.450000], [Loss: 0.396544] [Correct: 0.833000]

Epoch: 225
[Train: ], [225/250: ], [Accuracy: 98.810000], [Loss: 0.095857] [LossS: 0.000000] [Lr: 0.000010]
[Test Accuracy: 74.900000], [Loss: 0.394925] [Correct: 0.827500]

Epoch: 226
[Train: ], [226/250: ], [Accuracy: 98.840000], [Loss:

[Test Accuracy: 44.650000], [Loss: 0.484892] [Correct: 0.509000]
Saving..

Epoch: 17


In [ ]:
def inference_check_task(dtask,mtask,inputs,task_model):
    joint_entropy=[]
    with torch.no_grad():
        for m in range(mtask+1):
            task_model[m].eval()
            outputs,_=task_model[m](inputs)

            sout=F.softmax(outputs,1)
            entropy=-torch.sum(sout*torch.log(sout+0.0001),1)
            joint_entropy.append(entropy)


    all_entropy=torch.zeros([joint_entropy[0].shape[0],mtask+1]).cuda()
    for i in range(mtask+1):
        all_entropy[:,i]=joint_entropy[i]
    ctask=torch.argmin(all_entropy, axis=1)==dtask
    correct=sum(ctask)
    
    return ctask, correct,all_entropy



def inferecne(test_loader,dtask,mtask,task_model):
    global best_acc
    model=task_model[dtask].eval()
    test_loss = 0
    correct = 0
    total = 0
    cl_loss=0
    tcorrect=0
    accuracy=[]
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets1 = inputs.cuda(), targets.cuda()
            targets=targets1-dtask*args.class_per_task
                        
            if mtask>0:
                correct_sample,Ncorrect,_=inference_check_task(dtask,mtask,inputs,task_model)
                tcorrect+=Ncorrect
                inputs=inputs[correct_sample]
                targets=targets[correct_sample]
                
            if inputs.shape[0]!=0:
                outputs,_ = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()
            total += targets1.size(0)
    if mtask>0:
        taskC= tcorrect.item()/total
    else:
        taskC=1.0 
    acc = 100.*correct/total
    print("[Test Accuracy: %f], [Loss: %f] [Correct: %f]" %(acc,test_loss/(batch_idx+1),taskC))
    
    return acc,taskC

In [ ]:
cl_acc=[]
criterion = nn.CrossEntropyLoss()
for mtask in range(args.num_task):
    accuracy=[]
    corr=[]
    print('#######################################################')
    ttask=mtask+1
    for dtask in range(ttask):
        print('Testing Task :---'+str(dtask))
        test_loader = task_data[dtask][1]
        acc,correct=inferecne(test_loader,dtask,mtask,task_model)
        accuracy.append(acc)
        corr.append(correct)
    task_acc=np.mean(accuracy)
    task_correct=np.mean(corr)
    print('acc: '+str(task_acc)+'  Correct:  '+str(task_correct))
    cl_acc.append(task_acc)
print(np.array(cl_acc))